# Map Reduce

In [1]:
# formatting ONLY IN NOTEBOOK
import book_format
book_format.load_style()

## Procesamiento de información

### Un ejemplo
Tenemos un log de la siguiente manera

    2012-01-01 09:08 BOG Libros 88.56 Discover
    2012-01-01 09:52 BGA Libros 62.41 Discover
    2012-01-01 10:08 MED Musica 93.37 Visa
    2012-01-01 10:58 MED Musica 395.93 Discover
    2012-01-01 14:38 BOG Musica 113.24 MasterCard
    2012-01-01 14:44 BGA Libros 290.5 Visa
    2012-01-01 16:26 MED Musica 246.12 Efectivo

Con el `dia`, `hora`, `ciudad`, `producto`, `importe` y `medio de pago` de compras realizadas en almacenes de una cierta cadena. Queremos obtener el total del importe de compras realizadas en cada ciudad. Una forma de hacerlo es la siguiente:

1) procesamos cada linea y, de cada una, generamos un par `(ciudad, importe)`, obteniendo los siguientes pares:

    (BOG, 88.56)
    (BGA, 62.41)
    (MED, 93.37)
    (MED, 395.93)
    (BOG, 113.24)
    (BGA, 290.5)
    (MED, 246.12)
    
2) agrupamos las tuplas generadas por el valor del primer componente:

    (BOG, [88.56, 113.24])
    (BGA, [62.41, 290.5])
    (MED, [93.37, 395.93, 246.12])
     
3) sumamos los elementos de cada lista para cada ciudad:

    (BOG, 201.8)
    (BGA, 352.91)
    (MED, 735.42)
    



### Tres fases

El primer paso en el ejemplo anterior se denomina **MAP** y, para cada registro de entrada, genera una tupla de formato `(clave, valor)`.

El segundo paso se denomina **SHUFFLE** y lo que hace es recopilar todas las tuplas generadas en el MAP anterior de cada `clave` y construir una lista con todos los valores generados. Es decir, una tupla de formato `(clave, lista_de_valores)` para cada clave.

El tercer paso se denomina **REDUCE** y, para cada clave, agrega los resultados de la lista generada en el SHUFFLE anterior.

Esta forma de procesar los datos constituye un **modelo de programación** llamado *MAP-REDUCE* y que está implementado un varios frameworks de programación, de forma que el programador **solo programa las funciones MAP y REDUCE** y el shuffle está implementado en el framework.

### mr-job


Usaremos el framework [mr-job](https://pythonhosted.org/mrjob) para hacer nuestros programas map-reduce. El siguiente código implementa el proceso que acabamos de describir. Fíjate cómo sólo programamos las funciones `mapper` y `reducer`.

In [20]:
%%writefile files/mr-ejemplo.py
from mrjob.job import MRJob

class Compras(MRJob):

    def mapper(self, _, line):
        tokens = line.split()
        yield tokens[2], float(tokens[4])

    def reducer(self, key, values):
        yield key, sum(values)
        
if __name__ == '__main__':
    Compras.run()

Overwriting files/mr-ejemplo.py


In [21]:
%%script --err my_stderr python files/mr-ejemplo.py 
2012-01-01 09:08 BOG Libros 88.56 Discover
2012-01-01 09:52 BGA Libros 62.41 Discover
2012-01-01 10:08 MED Musica 93.37 Visa
2012-01-01 10:58 MED Musica 395.93 Discover
2012-01-01 14:38 BOG Musica 113.24 MasterCard
2012-01-01 14:44 BGA Libros 290.5 Visa
2012-01-01 16:26 MED Musica 246.12 Efectivo

"BGA"	352.90999999999997
"BOG"	201.8
"MED"	735.42


_Prueba a eliminar el término_ `--err my_stderr` _de la celda anterior y así verás los mensajes que emite **mr-job** durante la ejecución del programa_